In [ ]:
# 2 Exercises
# The file proj4_params.json contains a JSON dictionary with parameters which will be used
# throughout the exercises. Load it.

In [1]:
import json

params = json.load(open('proj4_params.json', 'r'))

In [3]:
# 2.1 Exercise 1: Loading data and basic operations
# 4 points (2 for each file)
# Load a set of points from a GeoJSON file called proj4_points.geojson. The points will always
# be located somewhere in Poland.
# Each point has a unique identifier in a columns specified by the id_column parameter loaded from
# the JSON file above (in the example dataset, it is lamp_id).
# For each point, count the number of points (including the point itself) that are within 100 metres
# of that point.
# Save the results to a file called proj4_ex01_counts.csv, with two columns:
# • the identifier column, with its original name,
# • a column called count with the number of “neighbouring” points.
# An example file could look like this:
# lamp_id,count
# 5907,16
# 5908,16
# 5909,17
# 5910,20
# 5911,9
# (...)
# 1
# Now save the latitude and longitude of all points to a CSV file called proj4_ex01_coords.csv,
# with the following columns:
# • the identifier column, with its original name,
# • lat for latitude,
# • lon for longitude.
# An example file could look like this:
# lamp_id,lat,lon
# 5907,50.07404343940157,19.899135469459004
# 5908,50.0750528346396,19.891393063589923
# 5909,50.07305532610415,19.898210107348856
# (...)

In [2]:
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

points = gpd.read_file('proj4_points.geojson')
id = params['id_column']
result = pd.DataFrame(columns=[id, 'count'])
result[id] = points[id]

points = points.to_crs(epsg = 2180)
points['buffer'] = points.geometry.buffer(100)

points['count'] = points.apply(lambda row: points.within(row['buffer']).sum(), axis = 1)

print(points[[id, 'count']])
points[[id, 'count']].to_csv('proj4_ex01_counts.csv', index = False)

points = points.to_crs(epsg = 4326)

points['lat'] = points.geometry.y
points['lon'] = points.geometry.x

print(points[[id, 'lat', 'lon']])
points[[id, 'lat', 'lon']].to_csv('proj4_ex01_coords.csv', index = False)



      lamp_id  count
0        5907     16
1        5908     16
2        5909     17
3        5910     20
4        5911      9
...       ...    ...
3737     9644     16
3738     9645     16
3739     9646     15
3740     9647     12
3741     9648     19

[3742 rows x 2 columns]
      lamp_id        lat        lon
0        5907  50.074043  19.899135
1        5908  50.075053  19.891393
2        5909  50.073055  19.898210
3        5910  50.073520  19.888718
4        5911  50.073280  19.891677
...       ...        ...        ...
3737     9644  50.073446  19.934272
3738     9645  50.070489  19.935150
3739     9646  50.070320  19.935348
3740     9647  50.070155  19.935541
3741     9648  50.067359  19.934254

[3742 rows x 3 columns]


In [4]:
# 2.2 Exercise 2: Loading data from OpenStreetMap
# 3 points
# The city parameter contains the name of the city where the points are located (e.g. Cracow). That
# city will be one of those available in Pyrosm, as well as one identifiable by OSMnx.
# Load OpenStreetMap data for that city into a GeoDataFrame. Only include drivable roads, and
# from those, only include primary ones, e.g. those with the highway key set to primary.
# Structure your GeoDataFrame so that it contains the following columns:
# • osm_id – the OpenStreetMap identifier of the street,
# • name – the name of the street,
# • geometry – the geometry.
# Save it to proj4_ex02_primary_roads.geojson.

In [ ]:
import pyrosm

fp = pyrosm.get_data(params['city'])
osm = pyrosm.OSM(fp)

roads = osm.get_network(network_type = 'driving')
primary_roads = roads[roads['highway'] == 'primary']
primary_roads['osm_id'] = primary_roads.index
primary_roads = primary_roads[['osm_id', 'name', 'geometry']]
primary_roads = primary_roads.reset_index(drop = True)

print(primary_roads)

primary_roads.to_file('proj4_ex02_primary_roads.geojson', driver = 'GeoJSON')

In [5]:
# 2.3 Exercise 3: Spatial joins
# 3 points
# For each of the roads obtained in Exercise 3, count the number of points, loaded in Exercise 1, that
# are within 50 metres of the line modelling the axis of the road.
# Save the results to a CSV file called proj4_ex03_streets_points.csv, with the following columns:
# • name, with the name of the street,
# • point_count, with the number of points within 50 metres of that street.
# Include streets with no points. If there are multiple OSM ways with the same street name, aggregate
# them. An example file could look as follows:
# name,point_count
# Aleja 29 Listopada,0
# Aleja Adama Mickiewicza,560
# Aleja Jana Pawła II,0
# Aleja Juliusza Słowackiego,394
# (...)

In [ ]:
points = points.to_crs(epsg = 2180)
primary_roads['buffer'] = primary_roads.geometry.buffer(50)
primary_roads['point_count'] = primary_roads.apply(lambda row: points.within(row['buffer']).sum(), axis = 1)
primary_roads = primary_roads.groupby('name')['point_count'].sum().reset_index()
primary_roads = primary_roads.sort_values(by = 'name')

print(primary_roads[['name', 'point_count']])
primary_roads[['name', 'point_count']].to_csv('proj4_ex03_streets_points.csv', index = False)

In [8]:
# 2.4 Exercise 4: Drawing maps
# 5 points (2 for GDF, 3 for images)
# The file proj4_countries.geojson contains polygons with boundaries of several countries from
# all over the World. Load the GeoJSON file into a GeoDataFrame.
# Each feature in the file has a property called name, which contains the name of the country.
# Modify the GeoDataFrame so that:
# • the boundaries are hollow, not filled,
# • the horizontal/vertical aspect ratios (proportions) of the shapes are correct.
# Please note that while we want the shapes to not be “squashed” either horizontally or vertically,
# it is acceptable for them to be have distortions which result from using Mercator projections. In
# other words, the shapes should look “good” on the map.
# Save the modified GeoDataFrame to proj4_ex04_gdf.pkl.
# Now render the boundary of each country to a separate PNG file, adding a background map to
# provide context.
# The name of the file should follow the scheme: proj4_ex04_COUNTRY.png, where COUNTRY is the
# country name in lowercase, e.g. proj4_ex03_poland.png, proj4_ex03_italy.png, etc.
# An example rendering could look like this:


In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx


countries = gpd.read_file('proj4_countries.geojson')
print(countries)

countries.plot(facecolor = 'none', edgecolor = 'black')
ctx.add_basemap(plt.gca(), crs = countries.crs.to_string())

countries.to_pickle('proj4_ex04_gdf.pkl')

for i, row in countries.iterrows():
    name = row['name'].lower()
    fig, ax = plt.subplots()
    geometry = gpd.GeoSeries(row.geometry)
    geometry.plot(ax = ax, facecolor = 'none', edgecolor = 'black')
    ctx.add_basemap(ax, crs = countries.crs.to_string())

    fig.savefig(f'proj4_ex04_{name}.png')